## **2025 Data Creator Camp**
같이DATA TEAM: Mission 4 (Data Analysis)

---

 이 ipynb는 학습 루프와 모델 구조에 집중하도록 설계되어져 있습니다. 이 ipynb에서 Hyperparameter로 사용하고 있는 데이터 별 각 채널의 정규화 통계값과 Oversampling 대상/비율은 'DCC_mission4_analysis_같이DATA.ipynb'에서 미리 계산하였습니다.

---

## **Google Mount**

Google Drive에 접근할 수 있도록 mount 합니다.

---

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **Import packages**

Mission을 수행하는데 있어서 요구되는 패키지 설치하고 불러옵니다.  
또한, Colab 상에서 모델 학습 시 **반복성을 보장**하기 위해 seed를 고정합니다.

In [ ]:
!pip install torchinfo
!pip install -U imagecodecs
!pip install segmentation-models-pytorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 122.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 13.4 MB/s eta 0:00:00


In [ ]:
import os
import glob
import random
from pathlib import Path
from typing import List, Dict, Any, Tuple
from datetime import datetime
import numpy as np

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from skimage import io, measure
import natsort
import imagecodecs
import tifffile

from tqdm import tqdm
from torchvision import transforms

## segmentation models
import segmentation_models_pytorch as smp
from segmentation_models_pytorch.encoders import get_encoder

## ViT
from transformers import ViTModel, ViTConfig
from transformers.models.vit.modeling_vit import ViTEncoder

## model info
from torchinfo import summary

## colab runtime
from google.colab import runtime

## data augmentation
import albumentations as A

## data oversampling
from torch.utils.data import Subset
from skimage import measure

In [ ]:
# ====== seed 고정 ======
def set_seed(seed=42):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True,  warn_only=True)
    os.environ["PYTHONHASHSEED"] = str(seed)
    os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
    torch.backends.cuda.enable_flash_sdp(False)
    torch.backends.cuda.enable_mem_efficient_sdp(False)

set_seed(42)
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(42)

## **Load data from Google Drive**

Google Drive에 저장된 **DCC 데이터셋**을 Google Colab 상으로 불러옵니다. 이는 데이터를 처리하고 학습시키는 속도를 단축시켜 줍니다.

---

### 데이터셋 구성

- **Train Image**: `TS_SN10_SN10`, `TS_SN10_GEMS`, `TS_SN10_AIR_Pollution`
- **Train Label**: `TL_SN10`  
- **Validation Image**: `VS_SN10_SN10`, `VS_SN10_GEMS`, `VS_SN10_AIR_Pollution`  
- **Validation Label**: `VL_SN10`

---


In [ ]:
!mkdir -p /content/Training/origin
!mkdir -p /content/Training/label
!mkdir -p /content/Validation/origin
!mkdir -p /content/Validation/label

## train
!cp /content/drive/MyDrive/DCC2025/Data/Training/01.원천데이터/TS_SN10_SN10.zip /content/
!cp /content/drive/MyDrive/DCC2025/Data/Training/01.원천데이터/TS_SN10_GEMS.zip /content/
!cp /content/drive/MyDrive/DCC2025/Data/Training/01.원천데이터/TS_SN10_AIR_Pollution.zip /content/
!cp /content/drive/MyDrive/DCC2025/Data/Training/02.라벨링데이터/TL_SN10.zip /content/

!unzip /content/TS_SN10_SN10.zip -d /content/Training/origin/TS_SN10_SN10
!unzip /content/TS_SN10_GEMS.zip -d /content/Training/origin/TS_SN10_GEMS
!unzip /content/TS_SN10_AIR_Pollution.zip -d /content/Training/origin
!unzip /content/TL_SN10.zip -d /content/Training/label/TL_SN10

## valid
!cp /content/drive/MyDrive/DCC2025/Data/Validation/01.원천데이터/VS_SN10_SN10.zip /content/
!cp /content/drive/MyDrive/DCC2025/Data/Validation/01.원천데이터/VS_SN10_GEMS.zip /content/
!cp /content/drive/MyDrive/DCC2025/Data/Validation/01.원천데이터/VS_SN10_AIR_Pollution.zip /content/
!cp /content/drive/MyDrive/DCC2025/Data/Validation/02.라벨링데이터/VL_SN10.zip /content/

!unzip /content/VS_SN10_SN10.zip -d /content/Validation/origin
!unzip /content/VS_SN10_GEMS.zip -d /content/Validation/origin/VS_SN10_GEMS
!unzip /content/VS_SN10_AIR_Pollution.zip -d /content/Validation/origin/VS_SN10_AIR_Pollution
!unzip /content/VL_SN10.zip -d /content/Validation/label/VL_SN10

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: /content/Validation/origin/VS_SN10_AIR_Pollution/AIR_Pollution_NO2_SN10_CHN_04813_230516.tif  
  inflating: /content/Validation/origin/VS_SN10_AIR_Pollution/AIR_Pollution_NO2_SN10_CHN_04886_231023.tif  
  inflating: /content/Validation/origin/VS_SN10_AIR_Pollution/AIR_Pollution_NO2_SN10_CHN_05020_230705.tif  
  inflating: /content/Validation/origin/VS_SN10_AIR_Pollution/AIR_Pollution_NO2_SN10_CHN_04650_240510.tif  
  inflating: /content/Validation/origin/VS_SN10_AIR_Pollution/AIR_Pollution_NO2_SN10_CHN_05072_231102.tif  
  inflating: /content/Validation/origin/VS_SN10_AIR_Pollution/AIR_Pollution_NO2_SN10_CHN_04908_231112.tif  
  inflating: /content/Validation/origin/VS_SN10_AIR_Pollution/AIR_Pollution_NO2_SN10_CHN_04807_230516.tif  
  inflating: /content/Validation/origin/VS_SN10_AIR_Pollution/AIR_Pollution_NO2_SN10_CHN_05284_240329.tif  
  inflating: /content/Validation/origin/VS_SN10_AIR_Pollution/AIR_Pollution_NO2_SN10_CHN_04975_23050

In [ ]:
base_dirs = {
    "Train_TS_SN10_SN10": "/content/Training/origin/TS_SN10_SN10",
    "Train_TS_SN10_GEMS": "/content/Training/origin/TS_SN10_GEMS",
    "Train_TS_SN10_AIR_Pollution": "/content/Training/origin/TS_SN10_AIR_Pollution",
    "Train_TL_SN10": "/content/Training/label/TL_SN10",
    "Valid_VS_SN10_SN10": "/content/Validation/origin/VS_SN10_SN10",
    "Valid_VS_SN10_GEMS": "/content/Validation/origin/VS_SN10_GEMS",
    "Valid_VS_SN10_AIR_Pollution": "/content/Validation/origin/VS_SN10_AIR_Pollution",
    "Valid_VL_SN10": "/content/Validation/label/VL_SN10",
}

# ====== 파일 개수 출력 ======
print("파일 개수 요약\n" + "="*40)
for name, path in base_dirs.items():
    if os.path.exists(path):
        files = glob.glob(os.path.join(path, "*.tif")) \
              + glob.glob(os.path.join(path, "*.png")) \
              + glob.glob(os.path.join(path, "*.jpg"))
        print(f"{name:<40}: {len(files)} files")
    else:
        print(f"{name:<40}: ❌ Path not found")

파일 개수 요약
Train_TS_SN10_SN10                      : 8000 files
Train_TS_SN10_GEMS                      : 8000 files
Train_TS_SN10_AIR_Pollution             : 24000 files
Train_TL_SN10                           : 8000 files
Valid_VS_SN10_SN10                      : 1000 files
Valid_VS_SN10_GEMS                      : 1000 files
Valid_VS_SN10_AIR_Pollution             : 3000 files
Valid_VL_SN10                           : 1000 files


## **Data Preprocessing**


---

아래 함수는 Train dataset 기반으로 데이터의 채널 별 min/max/mean/std를 구하는 코드입니다. 이는 16비트로 구성된 위성 영상 이미지를 효과적으로 정규화를 하기 위해 필수적입니다.

In [ ]:
# ====== 정규화 상수 정의 ======
'''
R/G/B/NIR/GEMS/AIR_POLLUTION의 정규화 상수는 DCC_mission4_anaysis_같이DATA.ipynb에서 계산되었습니다.
'''

## SN10 (Sentinel-2, 4채널: R,G,B,NIR) 정규화 상수
SN10_Q_MIN = np.array(
    [1045.7480, 1102.7969,  959.97656,  994.40234],
    dtype=np.float32
)
SN10_Q_MAX = np.array(
    [6740.9590, 6511.9140, 6216.43360, 7179.79100],
    dtype=np.float32
)

SN10_MEAN_01 = np.array(
    [0.18676323, 0.17453468, 0.16489673, 0.37120032],
    dtype=np.float32
)
SN10_STD_01 = np.array(
    [0.12096564, 0.10374451, 0.10795358, 0.17158176],
    dtype=np.float32
)

SN10_Q_MIN_T = torch.from_numpy(SN10_Q_MIN).view(4, 1, 1)
SN10_Q_MAX_T = torch.from_numpy(SN10_Q_MAX).view(4, 1, 1)
SN10_MEAN_01_T = torch.from_numpy(SN10_MEAN_01).view(4, 1, 1)
SN10_STD_01_T  = torch.from_numpy(SN10_STD_01).view(4, 1, 1)

## GEMS (12채널) 정규화 상수
GEMS_Q_MIN = np.array(
    [
        3.5162422e+15, 3.3423578e+15, 2.6562703e+15, 2.6152297e+15,
        3.5734262e+15, 4.5582300e+15, 5.6688098e+15, 5.7483848e+15,
        5.4511950e+15, 4.6075126e+15, 5.6982765e+15, 4.1897342e+15,
    ],
    dtype=np.float32
)
GEMS_Q_MAX = np.array(
    [
        1.9415181e+16, 1.5701159e+16, 1.1727013e+16, 1.1954129e+16,
        1.6976909e+16, 2.6186428e+16, 3.1782455e+16, 5.0354355e+16,
        4.1381615e+16, 2.7691782e+16, 2.9802686e+16, 2.1372565e+16,
    ],
    dtype=np.float32
)

GEMS_MEAN_01 = np.array(
    [
        0.40786100, 0.44841686, 0.38415930, 0.39808103,
        0.37259808, 0.48677856, 0.50863730, 0.42706522,
        0.50080620, 0.32017106, 0.41207560, 0.42404880,
    ],
    dtype=np.float32
)
GEMS_STD_01 = np.array(
    [
        0.16662924, 0.18663561, 0.18114029, 0.18778645,
        0.18000051, 0.22487241, 0.22179833, 0.21338297,
        0.23121008, 0.18217857, 0.19661322, 0.19239257,
    ],
    dtype=np.float32
)

GEMS_Q_MIN_T = torch.from_numpy(GEMS_Q_MIN).view(12, 1, 1)
GEMS_Q_MAX_T = torch.from_numpy(GEMS_Q_MAX).view(12, 1, 1)
GEMS_MEAN_01_T = torch.from_numpy(GEMS_MEAN_01).view(12, 1, 1)
GEMS_STD_01_T  = torch.from_numpy(GEMS_STD_01).view(12, 1, 1)

## AIR_Pollution (36채널 = NO2 12 + CO 12 + SO2 12) 정규화 상수
AIR_Q_MIN = np.array(
    [
        0.00490198, 0.00424099, 0.00323326, 0.00359276, 0.00512485, 0.00823938,
        0.00859833, 0.01010358, 0.00979392, 0.00506495, 0.00699119, 0.00568982,
        0.00213506, 0.00222388, 0.00206976, 0.00232351, 0.00256645, 0.00272072,
        0.00271155, 0.00366503, 0.00359954, 0.00291480, 0.00264298, 0.00229461,
        0.00114026, 0.00106933, 0.00088086, 0.00092519, 0.00103340, 0.00123184,
        0.00123002, 0.00134615, 0.00128073, 0.00121148, 0.00116172, 0.00114905,
    ],
    dtype=np.float32
)
AIR_Q_MAX = np.array(
    [
        0.01218081, 0.01031712, 0.00937196, 0.00882070, 0.01000861, 0.01385397,
        0.01518749, 0.01837835, 0.01792720, 0.01457242, 0.01324651, 0.01354300,
        0.34100255, 0.34722427, 0.33872578, 0.34639868, 0.35129570, 0.38673446,
        0.40784900, 0.47784310, 0.48674000, 0.43520680, 0.39244420, 0.38976192,
        0.00246800, 0.00238987, 0.00237140, 0.00239549, 0.00229022, 0.00242092,
        0.00254543, 0.00262165, 0.00275954, 0.00248687, 0.00254512, 0.00269394,
    ],
    dtype=np.float32
)

AIR_MEAN_01 = np.array(
    [
        0.19982699, 0.19986182, 0.19986899, 0.19984344, 0.19987953, 0.19986126,
        0.19990817, 0.19984410, 0.19986877, 0.19984931, 0.19986929, 0.19985457,
        0.19987690, 0.19987716, 0.19987687, 0.19987707, 0.19987708, 0.19987683,
        0.19987701, 0.19987565, 0.19987696, 0.19987679, 0.19987714, 0.19987687,
        0.19994368, 0.19991522, 0.19990437, 0.19993038, 0.19992624, 0.19997595,
        0.20002520, 0.19998385, 0.19998275, 0.19998807, 0.19994809, 0.19992930,
    ],
    dtype=np.float32
)
AIR_STD_01 = np.array(
    [
        0.39975417, 0.39979228, 0.39982742, 0.39977770, 0.39968783, 0.39961553,
        0.39969802, 0.39966822, 0.39969830, 0.39976308, 0.39971107, 0.39978594,
        0.39990526, 0.39990515, 0.39990530, 0.39990517, 0.39990517, 0.39990530,
        0.39990520, 0.39990250, 0.39990494, 0.39990523, 0.39990515, 0.39990530,
        0.39984950, 0.39983794, 0.39983743, 0.39985310, 0.39987767, 0.39983190,
        0.39982432, 0.39981666, 0.39983300, 0.39984754, 0.39986965, 0.39985620,
    ],
    dtype=np.float32
)

AIR_Q_MIN_T   = torch.from_numpy(AIR_Q_MIN).view(36, 1, 1)
AIR_Q_MAX_T   = torch.from_numpy(AIR_Q_MAX).view(36, 1, 1)
AIR_MEAN_01_T = torch.from_numpy(AIR_MEAN_01).view(36, 1, 1)
AIR_STD_01_T  = torch.from_numpy(AIR_STD_01).view(36, 1, 1)

## **Data Augmentation**  
Data Augmentation을 정의하는 부분입니다.  
Flip, rotation 등 geometry 적으로 증강시키는 transform과 noise, blur 등 이미지에만 적용시키는 transform을 다르게 적용합니다.


In [ ]:
def clip01(img, **kwargs):
    # 이미지는 0.0 ~ 1.0 범위로 클리핑
    return np.clip(img, 0.0, 1.0)

H, W = 512, 512 # SN10 및 Label의 해상도

transform_geometry = A.Compose([

    # A.VerticalFlip(p=0.2),
    # A.HorizontalFlip(p=1.0),
    A.RandomRotate90(p=0.4)

], additional_targets= {
    "air": "image",
    "gems": "image"},
is_check_shapes=False,)

transform_appearance = A.Compose([

    A.OneOf([
        A.GaussNoise(std_range=(0.0, 0.005), p=1.0),
        A.MultiplicativeNoise(multiplier=(0.95, 1.05), per_channel=True, p=1.0),
    ], p=0.3),

    A.Lambda(image=clip01) # Noise 추가 시 0~1 범위로 Clipping 필요

], additional_targets={'image': 'mask'})

## **Data loader**  
Colab 상으로 불러온 `TS_SN10`, `TS_SN10_GEMS`, `TS_SN10_AIR_Pollution` Data를 학습시 불러오는 **MultiModalDataset** class를 정의합니다.  
이때, Air pollution의 NO2, SO2, CO의 12 band는 합쳐져 36ch 형태로 불러옵니다.  



In [ ]:
class MultiModalDataset(Dataset):
    def __init__(self, base_dir, transform_geometry=None, transform_appearance=None, use_mean_std=False):
        """
        Colab으로 불러온 Multimodal dataset을 load 하고, Data augmentation을 적용합니다.
        """
        if os.path.normpath(base_dir).split(os.sep)[-1] == "Training":
            sub_title_data  = "origin/TS_SN10_"
            sub_title_label = "label/TL_SN10"
        else:
            sub_title_data  = "origin/VS_SN10_"
            sub_title_label = "label/VL_SN10"

        self.sn10_dir          = os.path.join(base_dir, sub_title_data + "SN10")
        self.air_pollution_dir = os.path.join(base_dir, sub_title_data + "AIR_Pollution")
        self.gems_dir          = os.path.join(base_dir, sub_title_data + "GEMS")
        self.label_dir         = os.path.join(base_dir, sub_title_label)
        self.transform_geometry         = None
        self.transform_appearance         = None
        self.use_mean_std      = use_mean_std

        self.file_list  = natsort.natsorted(os.listdir(self.sn10_dir))
        self.label_list = natsort.natsorted(os.listdir(self.label_dir))

        ## SN10: 4채널 (R,G,B,NIR)
        self.sn10_q_min  = SN10_Q_MIN.astype(np.float32).reshape(1, 1, 4)
        self.sn10_q_max  = SN10_Q_MAX.astype(np.float32).reshape(1, 1, 4)
        self.sn10_mean01 = SN10_MEAN_01.astype(np.float32).reshape(1, 1, 4)
        self.sn10_std01  = SN10_STD_01.astype(np.float32).reshape(1, 1, 4)

        ## GEMS: 12채널
        self.gems_q_min  = GEMS_Q_MIN.astype(np.float32).reshape(1, 1, 12)
        self.gems_q_max  = GEMS_Q_MAX.astype(np.float32).reshape(1, 1, 12)
        self.gems_mean01 = GEMS_MEAN_01.astype(np.float32).reshape(1, 1, 12)
        self.gems_std01  = GEMS_STD_01.astype(np.float32).reshape(1, 1, 12)

        ## AIR_Pollution: 36채널 (NO2 12 + CO 12 + SO2 12)
        self.air_q_min   = AIR_Q_MIN.astype(np.float32).reshape(1, 1, 36)
        self.air_q_max   = AIR_Q_MAX.astype(np.float32).reshape(1, 1, 36)
        self.air_mean01  = AIR_MEAN_01.astype(np.float32).reshape(1, 1, 36)
        self.air_std01   = AIR_STD_01.astype(np.float32).reshape(1, 1, 36)

    def __len__(self):
        return len(self.file_list)

    def _normalize_global(self, arr, q_min, q_max, mean01, std01):
        """
        arr   : (H,W,C) float32
        q_min : (1,1,C) 1% quantile
        q_max : (1,1,C) 99% quantile
        1) (x - q_min) / (q_max - q_min) -> 0~1 clip
        2) use_mean_std=True면 (x - mean01) / std01 까지 한다.
        """
        arr = arr.astype(np.float32)
        denom = (q_max - q_min)
        denom[denom == 0] = 1e-8

        x = (arr - q_min) / denom
        x = np.clip(x, 0.0, 1.0)

        if self.use_mean_std:
            x = (x - mean01) / std01

        return x

    def __getitem__(self, idx):
        fname = self.file_list[idx]

        # ====== SN10 (512x512x4) ======
        sn10_path = os.path.join(self.sn10_dir, fname)
        sn10 = io.imread(sn10_path).astype(np.float32)   # (H,W,4)
        sn10 = self._normalize_global(
            sn10,
            self.sn10_q_min,
            self.sn10_q_max,
            self.sn10_mean01,
            self.sn10_std01,
        )

        # ====== AIR_Pollution (NO2, CO, SO2 → 64x64x36) ======
        no2 = io.imread(os.path.join(self.air_pollution_dir, "AIR_Pollution_NO2_" + fname)).astype(np.float32)
        co  = io.imread(os.path.join(self.air_pollution_dir, "AIR_Pollution_CO_"  + fname)).astype(np.float32)
        so2 = io.imread(os.path.join(self.air_pollution_dir, "AIR_Pollution_SO2_" + fname)).astype(np.float32)
        air = np.concatenate([no2, co, so2], axis=2)    # (H,W,36)

        air = self._normalize_global(
            air,
            self.air_q_min,
            self.air_q_max,
            self.air_mean01,
            self.air_std01,
        )

        # ====== GEMS (64x64x12) ======
        gems_path = os.path.join(self.gems_dir, "GEMS_" + fname)
        gems = io.imread(gems_path).astype(np.float32)   # (H,W,12)

        gems = self._normalize_global(
            gems,
            self.gems_q_min,
            self.gems_q_max,
            self.gems_mean01,
            self.gems_std01,
        )

        # ====== Label (512x512) ======
        label_path = os.path.join(self.label_dir, self.label_list[idx])
        label = io.imread(label_path)
        # 10 → 1 (산업단지), 90 → 0 (배경)으로 Mapping
        label = np.where(label == 10, 1, label)
        label = np.where(label == 90, 0, label)
        label = label.astype(np.float32)

        # ====== 데이터 증강 적용 ======
        if self.transform_geometry:
            ## 기하학적 변환 적용 (sn10, air, gems, label 모두 변환)
            augmented_geometry = self.transform_geometry(
                image=sn10,
                air=air,
                gems=gems,
                mask=label,
            )
            sn10_geom_aug = augmented_geometry["image"]
            air         = augmented_geometry["air"]
            gems        = augmented_geometry["gems"]
            label       = augmented_geometry["mask"]
        else:
            sn10_geom_aug = sn10


        ## 외형적 증강 적용 (sn10에만 노이즈/블러 적용)
        if self.transform_appearance:
            augmented_appearance = self.transform_appearance(
                image=sn10_geom_aug,
            )
            sn10 = augmented_appearance["image"]
        else:
            sn10 = sn10_geom_aug

        # ====== Tensor 변환 ======
        sn10_t  = torch.from_numpy(sn10.transpose(2, 0, 1))   # (4,H,W)
        air_t   = torch.from_numpy(air.transpose(2, 0, 1))    # (36,h,w)
        gems_t  = torch.from_numpy(gems.transpose(2, 0, 1))   # (12,h,w)
        label_t = torch.from_numpy(label).unsqueeze(0)        # (1,H,W)

        return sn10_t, air_t, gems_t, label_t


## **Modified TransUNet**

TransUNet을 본 데이터셋에 맞춰 구조를 변형한 Modified TransUNet을 정의합니다.

---

### Pre-Trained Model
- **mit_b3**
- **ViT_B-16**

In [ ]:
class MultiConv(nn.Module):
    '''
    SN_10 input을 위한 Encoder에 활용되는 CNN 구조를 정의합니다.
    ReLU 대신 GELU를 활용하여 성능을 소폭 향상시켰습니다.
    '''
    def __init__(self, in_ch, out_ch, p_drop=0.0):
        super().__init__()
        '''
        same feature map size, change channel
        '''
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.GELU(),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.GELU(),
        )

        self.drop = nn.Dropout2d(p_drop) if p_drop > 0 else nn.Identity()

    def forward(self, x):
        x = self.conv(x)
        return self.drop(x)

class PatchEncoder(nn.Module):
    '''
    GEMS, AIR_POLLUTION Input을 위한 Encoder를 정의합니다.
    각 Input의 차원에 따라 Encoder를 정의하고, pre-trained model을 활용하여 Feature를 추출합니다.
    이때, ViT의 입력으로 들어가기 이전 Upsampling을 통해 size를 맞춰줍니다.
    '''
    def __init__(self, in_ch, base_ch, target_size):
        super().__init__()

        self.encoder = get_encoder(
            name="mit_b3",
            in_channels=in_ch,   # GEMS: 12, Air: 36
            depth=4,
            weights="imagenet",
        )

        enc_ch = self.encoder.out_channels[-1]
        self.adapter = nn.Conv2d(enc_ch, base_ch * 16, kernel_size=1)

        ## Output size 통일
        self.upsample = nn.Upsample(
            size=(target_size, target_size),
            mode="bilinear",
            align_corners=False
        )

    def forward(self, x):
        features = self.encoder(x)
        f = features[-1]               # 마지막 stage 출력 (B, enc_ch, H, W)
        f = self.adapter(f)            # (B, base_ch*16, H, W)
        f = self.upsample(f)           # (B, base_ch*16, target_size, target_size) = (B, base_ch*16, 32, 32)
        return f

def resize_pos_embed(pos_embed: torch.Tensor,
                          new_h: int, new_w: int) -> torch.Tensor:
        """
        ViT의 1D position_embeddings (cls + patch들)을 토큰 해상도(new_h x new_w)에 맞게 2D interpolation 합니다.
        """

        cls_pos  = pos_embed[:, 0:1, :]
        patch_pos = pos_embed[:, 1:, :]

        num_patches = patch_pos.shape[1]
        orig_size = int(num_patches ** 0.5)   # 196 → 14
        assert orig_size * orig_size == num_patches, "ViT patch pos_embed가 정사각형이 아니라서 리사이즈 불가하다."

        patch_pos = patch_pos.reshape(1, orig_size, orig_size, -1)   # (1, 14, 14, C)
        patch_pos = patch_pos.permute(0, 3, 1, 2)                    # (1, C, 14, 14)

        patch_pos = F.interpolate(
            patch_pos,
            size=(new_h, new_w),
            mode="bicubic",
            align_corners=False,
        )

        patch_pos = patch_pos.permute(0, 2, 3, 1).reshape(1, new_h*new_w, -1)
        return patch_pos

class MultiModalTransUNet_ViTpretrained(nn.Module):
    '''
    Modified TransUNet을 정의합니다.
    '''
    def __init__(self, n_classes=2, N=32, vit_name="google/vit-base-patch16-224", freeze_vit=False, use_sincos_pe=True, pe_drop=0.0):
        super().__init__()

        self.enc1 = MultiConv(4, N)

        self.smp_encoder = get_encoder(
            name="mit_b3",
            in_channels=4,
            depth=4,
            weights="imagenet",
        )

        enc_ch = self.smp_encoder.out_channels

        self.adapter_x3 = nn.Conv2d(enc_ch[2], N*4, 1)
        self.adapter_x4 = nn.Conv2d(enc_ch[3], N*8, 1)
        self.adapter_x5 = nn.Conv2d(enc_ch[4], N*16, 1)

        # ====== Multimodal Encoder for GEMS, AIR_POLLUTION ======
        self.patch1 = PatchEncoder(36, N, target_size=32)
        self.patch2 = PatchEncoder(12, N, target_size=32)

        # ====== ViT ======
        vit_full_model = ViTModel.from_pretrained(vit_name, hidden_dropout_prob = 0.1)
        self.vit_encoder = vit_full_model.encoder

        config = vit_full_model.config

        if freeze_vit:
            for p in self.vit_encoder.parameters():
                p.requires_grad = False

        print(f"[ViT] loaded '{vit_name}', hidden={config.hidden_size}, "
              f"trainable={not freeze_vit} 한다.")

        self.proj = nn.Conv2d(N*16*3, config.hidden_size, kernel_size=1)

        vit_pos_embed = vit_full_model.embeddings.position_embeddings
        pos_2d = resize_pos_embed(vit_pos_embed, new_h=32, new_w=32)

        self.register_buffer("pos_embed_2d", pos_2d, persistent=False)
        self.pe_drop = nn.Dropout(pe_drop) if pe_drop > 0 else nn.Identity()
        self.pe_scale = nn.Parameter(torch.tensor(1.0))

        # ====== Decoder ======
        self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False)
        self.dec4 = MultiConv(config.hidden_size + N*8, N*8)
        self.dec3 = MultiConv(N*8 + N*4, N*4)
        self.dec2 = MultiConv(N*4, N*2)

        self.dec1 = MultiConv(N*2 + N, N)
        self.final = nn.Conv2d(N, n_classes, kernel_size=1)

    def forward(self, x_img, x_patch1, x_patch2):
        # ====== Encoder ======
        x1 = self.enc1(x_img)
        features = self.smp_encoder(x_img)
        x3 = self.adapter_x3(features[2])
        x4 = self.adapter_x4(features[3])
        x5 = self.adapter_x5(features[4])  # (B, 512, 32, 32)

        ## ViT의 입력으로 차원을 맞춰줘야함
        if x5.shape[2:] != (32, 32):
            x5 = F.interpolate(x5, size=(32, 32), mode="bilinear", align_corners=False)

        # ====== Patch Enocder ======
        p1 = self.patch1(x_patch1)
        p2 = self.patch2(x_patch2)

        # ====== Concatenate & Projection ======
        combined = torch.cat([x5, p1, p2], dim=1)  # (B,1536,32,32)
        x = self.proj(combined)                    # (B,768,32,32)
        B, C, H, W = x.shape

        ## Flatten → ViT input
        x = x.flatten(2).transpose(1, 2)  # (B, 1024, 768)

        # ====== ViT ======
        ## ViT pretrained positional embedding 추가
        x = x + self.pe_scale * self.pos_embed_2d
        x = self.pe_drop(x)

        vit_out = self.vit_encoder(hidden_states=x).last_hidden_state  # (B, 1024, 768)

        # 복원
        x = vit_out.transpose(1, 2).view(B, C, H, W)  # (B, 768, 32, 32) (C=config.hidden_size)

        # ====== Decoder ======
        ##  SN_10의 RGBNIR에서 추출한 Feature를 차원에 맞게 Skip 해 줍니다.
        d4 = self.up(x)
        d4 = self.dec4(torch.cat([d4, x4], dim=1))
        d3 = self.up(d4)
        d3 = self.dec3(torch.cat([d3, x3], dim=1))
        d2 = self.up(d3)
        d2 = self.dec2(d2)
        d1 = self.up(d2)
        d1 = self.dec1(torch.cat([d1, x1], dim=1))
        out = self.final(d1)
        return out

## **Functions**  

 평가 지표인 mIoU를 구현하고 기타 함수를 구현합니다.

---

In [ ]:
@torch.no_grad()
def batch_inter_union_2class(logits, masks, num_classes=2):
    """
    logits : (B, 2, H, W) - 모델 출력 (logits)
    masks  : (B, H, W) 또는 (B,1,H,W) - GT class {0,1}

    return:
        inter: (C,) 각 클래스별 intersection
        union: (C,) 각 클래스별 union
    """

    if masks.ndim == 4:
        masks = masks.squeeze(1)
    masks = masks.long()

    preds = torch.argmax(logits, dim=1)

    inter = torch.zeros(num_classes, dtype=torch.float64, device=logits.device)
    union = torch.zeros(num_classes, dtype=torch.float64, device=logits.device)

    for c in range(num_classes):
        p = (preds == c)
        t = (masks == c)
        inter[c] = (p & t).sum()
        union[c] = (p | t).sum()

    return inter, union


def iou_from_stats(inter, union):
    """
    inter, union: (C,) 누적 intersection / union

    return:
        miou           : 전체 mIoU (float)
        iou_per_class  : (C,) 클래스별 IoU 텐서
    """
    eps = 1e-7
    iou_per_class = inter / (union + eps)

    valid = union > 0
    if valid.any():
        miou = iou_per_class[valid].mean().item()
    else:
        miou = 0.0

    return miou, iou_per_class

@torch.no_grad()
def evaluate_2class_miou(model, loader, device, name="Val", num_classes=2):
    """
    학습 시와 동일한 방식(2-class IoU)으로 mIoU를 계산하는 함수
    """
    model.eval()

    inter_all = torch.zeros(num_classes, dtype=torch.float64, device=device)
    union_all = torch.zeros(num_classes, dtype=torch.float64, device=device)

    pbar = tqdm(loader, desc=f"Eval mIoU ({name})")
    for sn10, air, gems, label in pbar:
        sn10  = sn10.to(device, non_blocking=True)
        air   = air.to(device, non_blocking=True)
        gems  = gems.to(device, non_blocking=True)
        label = label.to(device, non_blocking=True)

        logits = model(sn10, air, gems)

        inter, union = batch_inter_union_2class(
            logits, label, num_classes=num_classes
        )
        inter_all += inter
        union_all += union

    miou, iou_per_class = iou_from_stats(inter_all, union_all)
    log_line(
        f"[{name}] 2-class mIoU={miou:.4f}, IoU per class={iou_per_class.tolist()} 한다."
    )
    return miou, iou_per_class, inter_all, union_all

In [ ]:
# ====== save_dir 설정 ======
BASE = Path("/content/drive/MyDrive")
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
save_dir = os.path.join(BASE / "runs", timestamp)
os.makedirs(save_dir, exist_ok=True)
print("저장 경로: ", save_dir)

# ====== log.txt 설정 ======
log_path = os.path.join(save_dir, "log.txt")

def log_line(text: str):
    """
    log.txt에 한 줄 기록 + 같은 포맷으로 콘솔에도 출력한다.
    [HH:MM:SS.mmm] text
    """
    ts = datetime.now().strftime("%H:%M:%S.%f")[:-3]  # 밀리초까지
    line = f"[{ts}] {text}"
    # 파일에 기록
    with open(log_path, "a", encoding="utf-8") as f:
        f.write(line + "\n")
    # 콘솔에도 출력
    print(line)
    return line
log_line(f"🚀 Training started. Logs will be saved to {log_path}")

# ========= set device =========
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

저장 경로:  /content/drive/MyDrive/runs/2025-11-25_05-11-53
[05:11:53.459] 🚀 Training started. Logs will be saved to /content/drive/MyDrive/runs/2025-11-25_05-11-53/log.txt
Using device: cuda


## **Data Oversampling**  

 현재 데이터는 산업단지 영역이 매우 작게 나타나는 **클래스 불균형** 현상이 발생하고 있습니다. 특히 작은 산업단지 클러스터가 포함된 이미지는 개수 자체가 적어 학습에 자주 사용되고 있지 않습니다. 이대로 학습에 사용하게 되면 모델이 배경 위주로 예측하게 되므로 Data Oversampling을 사용하여 클래스 불균형을 해소합니다. <br>

---

- **Base Model Weight**: `/content/drive/MyDrive/runs/base_model.pt`

---

In [ ]:
# ====== data loader ======
batch_size = 16

TRAIN_BASE = "/content/Training"
VAL_BASE   = "/content/Validation"

train_dataset = MultiModalDataset(base_dir=TRAIN_BASE, transform_geometry=None, transform_appearance=None)
val_dataset   = MultiModalDataset(base_dir=VAL_BASE, transform_geometry=None, transform_appearance=None)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4,  worker_init_fn=seed_worker, generator=g)
val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4,  worker_init_fn=seed_worker, generator=g)

print(f"Train samples: {len(train_dataset)} | Val samples: {len(val_dataset)}")

Train samples: 8000 | Val samples: 1000


In [ ]:
def build_oversampled_train_dataset(
    train_dataset,
    posratio_threshold: float = 0.3,
    oversample_threshold: int = 700,
    oversample_ratio: int = 3,
):
    """
    train_dataset        : MultiModalDataset 객체
    posratio_threshold   : 증폭할 positive ratio 기준
    oversample_threshold : 증폭할 클러스터 면적 기준
    oversample_ratio     : 후보 이미지를 증폭할 비율

    return:
        oversampled_train_dataset : Subset(MultiModalDataset, oversampled_indices)
        oversampled_indices       : np.ndarray, 실제로 사용할 인덱스 리스트
    """

    label_dir   = train_dataset.label_dir
    label_files = train_dataset.label_list
    file_list   = train_dataset.file_list

    num_imgs = len(file_list)
    if num_imgs != len(label_files):
        raise ValueError(f"이미지 수({num_imgs})와 라벨 수({len(label_files)})가 다르다 한다.")

    ## OPR와 Group A 대상 인덱스 계산
    pos_array  = np.zeros(num_imgs, dtype=np.float32)
    A_indices  = []  # OPR<=thr & 작은 클러스터 우세한 이미지 인덱스

    for idx, lab_name in enumerate(label_files):
        label_path = os.path.join(label_dir, lab_name)

        ## 라벨 로드 및 이진화 (10 → 1, 90/128 → 0) 한다.
        mask = io.imread(label_path)
        mask = np.where(mask == 10, 1, mask)
        mask = np.where((mask == 90) | (mask == 128), 0, mask)
        mask = (mask == 1).astype(np.uint8)  # 0/1

        ## OPR (양성 픽셀 비율 = positive ration)
        pos_ratio = mask.mean()
        pos_array[idx] = pos_ratio

        ## OPR 기준에 걸리지 않으면 oversampling 후보에서 제외
        if pos_ratio > posratio_threshold:
            continue

        ## 연결 성분 레이블링
        labeled = measure.label(mask, connectivity=1)
        props = measure.regionprops(labeled)

        if len(props) > 0:
            areas = np.array([p.area for p in props], dtype=np.int32)
            small_cnt = (areas <= oversample_threshold).sum()
            big_cnt   = (areas >  oversample_threshold).sum()
        else:
            small_cnt = big_cnt = 0

        ## 작은 클러스터 수가 더 많은 이미지 → Group A → oversampling 대상
        if small_cnt > big_cnt:
            A_indices.append(idx)

    low_indices = np.array(A_indices, dtype=np.int64)
    low_count   = int(low_indices.size)

    log_line("")
    log_line("===== Oversampling target (Train, cluster-based) =====")
    log_line(f"[Train] OPR <= {posratio_threshold:.2f} & area <= {oversample_threshold} 에서")
    log_line(f"[Train] 작은 클러스터 수가 더 많은 Group A 이미지 수: {low_count}개 한다.")
    log_line("[Train] 해당 이미지 파일 이름 목록:")

    for idx in low_indices:
        fname = file_list[idx]
        log_line(f"  - {fname}")

    # ====== Oversampled train datset 정의 ======
    orig_indices = np.arange(len(train_dataset), dtype=np.int64)

    if low_count > 0 and oversample_ratio > 0:
        extra_indices = np.repeat(low_indices, oversample_ratio)
        oversampled_indices = np.concatenate([orig_indices, extra_indices])
    else:
        oversampled_indices = orig_indices

    log_line("")
    orig_train_n  = len(orig_indices)
    extra_samples = len(oversampled_indices) - orig_train_n
    new_total_n   = len(oversampled_indices)

    log_line(f"[Train] 원본 train 개수: {orig_train_n}개 한다.")
    log_line(f"[Train] oversample 대상 {low_count}개 × ratio {oversample_ratio} → 추가 {extra_samples}개 샘플 된다.")
    log_line(f"[Train] 오버샘플링 이후 이론상 전체 train 개수: {new_total_n}개 한다.")
    log_line("========================================")

    oversampled_train_dataset = Subset(train_dataset, oversampled_indices.tolist())
    return oversampled_train_dataset, oversampled_indices

In [ ]:
# ====== Oversampled Train Dataset Generation ======
oversampled_train_dataset, oversampled_indices = build_oversampled_train_dataset(
    train_dataset,
    posratio_threshold=0.3,
    oversample_threshold=700,
    oversample_ratio=3,
)

# ====== Oversampled Train Dataset Definition ======
oversampled_train_loader = DataLoader(
    oversampled_train_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4,
    worker_init_fn=seed_worker,
    generator=g,
)

[05:12:45.122] 
[05:12:45.125] ===== Oversampling target (Train, cluster-based) =====
[05:12:45.128] [Train] OPR <= 0.30 & area <= 700 에서
[05:12:45.130] [Train] 작은 클러스터 수가 더 많은 Group A 이미지 수: 343개 한다.
[05:12:45.132] [Train] 해당 이미지 파일 이름 목록:
[05:12:45.134]   - SN10_CHN_00016_230409.tif
[05:12:45.136]   - SN10_CHN_00026_230409.tif
[05:12:45.138]   - SN10_CHN_00051_230409.tif
[05:12:45.140]   - SN10_CHN_00057_230409.tif
[05:12:45.142]   - SN10_CHN_00089_230514.tif
[05:12:45.144]   - SN10_CHN_00124_230514.tif
[05:12:45.146]   - SN10_CHN_00130_230514.tif
[05:12:45.148]   - SN10_CHN_00172_230608.tif
[05:12:45.150]   - SN10_CHN_00197_230608.tif
[05:12:45.152]   - SN10_CHN_00235_231021.tif
[05:12:45.154]   - SN10_CHN_00245_231021.tif
[05:12:45.156]   - SN10_CHN_00276_231021.tif
[05:12:45.158]   - SN10_CHN_00308_240319.tif
[05:12:45.160]   - SN10_CHN_00336_240319.tif
[05:12:45.176]   - SN10_CHN_00349_240319.tif
[05:12:45.179]   - SN10_CHN_00381_240508.tif
[05:12:45.182]   - SN10_CHN_00383_24050

## **Define variables**  

 data loader, model 등 학습에 요구되는 변수들을 정의합니다.<br>

---

In [ ]:
# ====== data loader ======
batch_size = 16

train_loader = DataLoader(oversampled_train_dataset, batch_size=batch_size, shuffle=True, num_workers=4,  worker_init_fn=seed_worker, generator=g)
val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4,  worker_init_fn=seed_worker, generator=g)

print(f"Train samples: {len(oversampled_train_dataset)} | Val samples: {len(val_dataset)}")

Train samples: 9029 | Val samples: 1000


In [ ]:
# ====== model ======
set_seed(42)
model =  MultiModalTransUNet_ViTpretrained(n_classes=2).to(device)

summary(
    model,
    input_size=[(1, 4, 512, 512),    # RGBNIR
                (1, 36, 64, 64),     # Air pollution
                (1, 12, 64, 64)],    # GEMS
    col_names=("input_size","output_size","num_params","kernel_size","mult_adds"),
    depth=3,
    device=device
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/135 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/178M [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[ViT] loaded 'google/vit-base-patch16-224', hidden=768, trainable=True 한다.


Layer (type:depth-idx)                             Input Shape               Output Shape              Param #                   Kernel Shape              Mult-Adds
MultiModalTransUNet_ViTpretrained                  [1, 4, 512, 512]          [1, 2, 512, 512]          1                         --                        --
├─MultiConv: 1-1                                   [1, 4, 512, 512]          [1, 32, 512, 512]         --                        --                        --
│    └─Sequential: 2-1                             [1, 4, 512, 512]          [1, 32, 512, 512]         --                        --                        --
│    │    └─Conv2d: 3-1                            [1, 4, 512, 512]          [1, 32, 512, 512]         1,184                     [3, 3]                    310,378,496
│    │    └─BatchNorm2d: 3-2                       [1, 32, 512, 512]         [1, 32, 512, 512]         64                        --                        64
│    │    └─GELU: 3-3               

## **Set hyper parameters for train**  

모델 학습에 요구되는 hyper parameter를 정의합니다.<br>

---

- **Learning rate**: `3e-4`
- **Epochs**: `50`
- **Loss Function**: `DICE(0.4) + CrossEntropy(0.6)`
- **Optimizer**: `AdamW`
- **Scheduler**: `CosineAnnealingLR`

In [ ]:
# ====== hyperparameteres ======
learning_rate = 3e-4
epochs = 50

# ====== Loss / Optimizer / Scheduler ======
dice_loss = smp.losses.DiceLoss(mode = "multiclass", from_logits=True)
ce_loss = nn.CrossEntropyLoss()

def loss_fn(preds, targets):
    return 0.4 * dice_loss(preds, targets) + 0.6 * ce_loss(preds, targets)

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs, eta_min=1e-6)

## **Train Loop**  

모델 학습을 진행합니다.<br>

---

In [ ]:
# ====== Training ======
scaler = torch.amp.GradScaler("cuda")
best_val_miou = float('-inf')

# ====== List for log ======
train_losses, val_losses = [], []
train_mious, val_mious = [], []

for epoch in range(epochs):

    model.train()
    running_loss, running_miou = 0.0, 0.0

    inter_train = torch.zeros(2, dtype=torch.float64, device=device)
    union_train = torch.zeros(2, dtype=torch.float64, device=device)

    ## tqdm으로 진행률 표시
    pbar = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{epochs}] (Train)", leave=True)
    for sn10, air, gems, label in pbar:
        sn10, air, gems, label = sn10.to(device), air.to(device), gems.to(device), label.squeeze(1).long().to(device)

        optimizer.zero_grad()

        with torch.amp.autocast("cuda"):
            outputs = model(sn10, air, gems)
            loss = loss_fn(outputs, label)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item()

        inter_b, union_b = batch_inter_union_2class(outputs, label, num_classes=2)
        inter_train += inter_b
        union_train += union_b

        train_miou_sofar, _ = iou_from_stats(inter_train, union_train)

        current_loss = running_loss / (pbar.n + 1)
        pbar.set_postfix({
            "loss": f"{current_loss:.4f}",
            "mIoU": f"{train_miou_sofar:.4f}"
        })

    train_loss = running_loss / len(train_loader)

    train_miou, train_iou_per_class = iou_from_stats(inter_train, union_train)
    train_bg_iou = train_iou_per_class[0].item()   # 클래스 0 (배경)
    train_fg_iou = train_iou_per_class[1].item()   # 클래스 1 (산업단지)

    # ====== Validation ======
    model.eval()
    val_loss, val_miou = 0.0, 0.0
    inter_val = torch.zeros(2, dtype=torch.float64, device=device)
    union_val = torch.zeros(2, dtype=torch.float64, device=device)

    with torch.no_grad():
        vbar = tqdm(val_loader, desc=f"Epoch [{epoch+1}/{epochs}] (Val)", leave=True)
        for sn10, air, gems, label in vbar:
            sn10, air, gems, label = sn10.to(device), air.to(device), gems.to(device), label.squeeze(1).long().to(device)

            outputs = model(sn10, air, gems)
            loss = loss_fn(outputs, label)
            val_loss += loss.item()

            inter_b, union_b = batch_inter_union_2class(outputs, label, num_classes=2)
            inter_val += inter_b
            union_val += union_b

            val_miou_sofar, _ = iou_from_stats(inter_val, union_val)
            vbar.set_postfix({
                "loss": f"{val_loss/(vbar.n+1):.4f}",
                "mIoU": f"{val_miou_sofar:.4f}"
            })

    val_loss /= len(val_loader)
    val_miou, val_iou_per_class = iou_from_stats(inter_val, union_val)
    val_bg_iou = val_iou_per_class[0].item()
    val_fg_iou = val_iou_per_class[1].item()

    train_losses.append(train_loss); val_losses.append(val_loss)
    train_mious.append(train_miou); val_mious.append(val_miou)

    msg = (
        f"[{epoch+1:02d}/{epochs}] "
        f"TrainLoss={train_loss:.4f}, "
        f"Train mIoU={train_miou:.4f} "
        f"(bg={train_bg_iou:.4f}, fg={train_fg_iou:.4f}) | "
        f"ValLoss={val_loss:.4f}, "
        f"Val mIoU={val_miou:.4f} "
        f"(bg={val_bg_iou:.4f}, fg={val_fg_iou:.4f})"
    )

    log_line(msg)
    print(msg)

    # ====== Scheduler step ======
    scheduler.step()

    # ====== Save best model ======
    if best_val_miou < val_miou:
        best_val_miou = val_miou
        torch.save(model.state_dict(), f"{save_dir}/best_model.pt")
        log_line(f"✅ Best model saved (val_miou={best_val_miou:.4f})")

    torch.cuda.empty_cache()

# ====== 그래프 저장 ======
plt.figure()
plt.plot(train_mious, label='Train mIoU')
plt.plot(val_mious, label='Val mIoU')
plt.xlabel('Epoch')
plt.ylabel('mIoU')
plt.legend(loc='lower right')
plt.title('Training vs Validation mIoU')
plt.savefig(f'{save_dir}/training_miou.png', dpi=300)
plt.close()

plt.figure()
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.title('Training vs Validation Loss')
plt.savefig(f'{save_dir}/training_loss.png', dpi=300)
plt.close()

print("Saved training_miou.png and training_loss.png")

pd.DataFrame({
    "train_loss": train_losses,
    "val_loss": val_losses,
    "train_miou": train_mious,
    "val_miou": val_mious
}).to_csv(f"{save_dir}/training_history.csv", index=False)

print("Saved training_history.csv")
log_line("Training finished.")

Epoch [1/50] (Train):   0%|          | 0/565 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/nn/functional.py:3458: UserWarning: nll_loss2d_forward_out_cuda_template does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:157.)
  return torch._C._nn.cross_entropy_loss(
Epoch [1/50] (Val): 100%|██████████| 63/63 [00:41<00:00,  1.54it/s, loss=0.0884, mIoU=0.8993]


[05:32:12.987] [01/50] TrainLoss=0.1617, Train mIoU=0.8389 (bg=0.9411, fg=0.7368) | ValLoss=0.0884, Val mIoU=0.8993 (bg=0.9631, fg=0.8355)
[01/50] TrainLoss=0.1617, Train mIoU=0.8389 (bg=0.9411, fg=0.7368) | ValLoss=0.0884, Val mIoU=0.8993 (bg=0.9631, fg=0.8355)
[05:32:14.941] ✅ Best model saved (val_miou=0.8993)


Epoch [2/50] (Val): 100%|██████████| 63/63 [00:36<00:00,  1.71it/s, loss=0.0654, mIoU=0.9190]


[05:51:15.047] [02/50] TrainLoss=0.0676, Train mIoU=0.9151 (bg=0.9709, fg=0.8593) | ValLoss=0.0654, Val mIoU=0.9190 (bg=0.9708, fg=0.8671)
[02/50] TrainLoss=0.0676, Train mIoU=0.9151 (bg=0.9709, fg=0.8593) | ValLoss=0.0654, Val mIoU=0.9190 (bg=0.9708, fg=0.8671)
[05:51:17.341] ✅ Best model saved (val_miou=0.9190)


Epoch [3/50] (Val): 100%|██████████| 63/63 [00:37<00:00,  1.70it/s, loss=0.0481, mIoU=0.9377]


...

Epoch [49/50] (Val): 100%|██████████| 63/63 [00:36<00:00,  1.71it/s, loss=0.0164, mIoU=0.9831]


[20:48:51.748] [49/50] TrainLoss=0.0049, Train mIoU=0.9927 (bg=0.9976, fg=0.9878) | ValLoss=0.0164, Val mIoU=0.9831 (bg=0.9941, fg=0.9721)
[49/50] TrainLoss=0.0049, Train mIoU=0.9927 (bg=0.9976, fg=0.9878) | ValLoss=0.0164, Val mIoU=0.9831 (bg=0.9941, fg=0.9721)
[20:48:53.795] ✅ Best model saved (val_miou=0.9831)


Epoch [50/50] (Val): 100%|██████████| 63/63 [00:36<00:00,  1.71it/s, loss=0.0164, mIoU=0.9831]


[21:07:54.253] [50/50] TrainLoss=0.0049, Train mIoU=0.9927 (bg=0.9976, fg=0.9879) | ValLoss=0.0164, Val mIoU=0.9831 (bg=0.9941, fg=0.9721)
[50/50] TrainLoss=0.0049, Train mIoU=0.9927 (bg=0.9976, fg=0.9879) | ValLoss=0.0164, Val mIoU=0.9831 (bg=0.9941, fg=0.9721)
[21:07:56.289] ✅ Best model saved (val_miou=0.9831)
Saved training_miou.png and training_loss.png
Saved training_history.csv
[21:07:59.566] Training finished.


'[21:07:59.566] Training finished.'

In [ ]:
runtime.unassign()